In [1]:
import pandas as pd
import numpy as np
import os
from PIL import Image as PImage
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import torch
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from typing import List
%matplotlib inline

In [2]:
DATA_FOLDER = './train'

In [3]:
train_df_ = pd.read_csv('./train.csv')
train_df_[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']] = train_df_[['grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].astype('uint8')
train_df_.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


Prepare dataset 

In [4]:
for i in range(4):
    train_df = pd.merge(pd.read_parquet(os.path.join(DATA_FOLDER, 'train_image_data_{}.parquet'.format(i))),
                        train_df_, on='image_id').drop(['image_id'], axis=1)
#to image
train_labels = train_df[['grapheme_root','vowel_diacritic','consonant_diacritic','grapheme']]
train_df.drop(['grapheme_root','vowel_diacritic','consonant_diacritic','grapheme'], axis=1, inplace=True)

Split into folds

In [5]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
train_labels['vowel_consonant_diacritic_pair'] = train_labels['vowel_diacritic'].astype(str) + '_' + train_labels['consonant_diacritic'].astype(str)
train_labels['test_fold'] = 0 
for idx, (train_index, test_index) in enumerate(skf.split(X = train_labels['vowel_consonant_diacritic_pair'], 
                                                          y = train_labels['vowel_consonant_diacritic_pair'], 
                                                          groups=train_labels['vowel_consonant_diacritic_pair'])):
    train_labels['test_fold'][test_index] = idx

/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pan

Prepare Callbacks and Dataset for training

In [72]:
class ImageDataset(Dataset):
    def __init__(self, 
                 df, 
                 labels,
                 transforms=None):
        
        self.df = df
        self.labels = labels
        self.transforms = transforms
        
    def __getitem__(self, idx):
        flattened_image = self.df.iloc[idx].values.astype(np.uint8)
        image = np.expand_dims(flattened_image.reshape(137, 236), 2)
        
        grapheme_root =  self.labels['grapheme_root'].values[idx]
        vowel_diacritic = self.labels['vowel_diacritic'].values[idx]
        consonant_diacritic = self.labels['consonant_diacritic'].values[idx]
        
        if self.transforms is not None:
            augmented = self.transforms(image=image)
            image = augmented['image']
        
        image = torch.from_numpy(image.transpose((2,0,1)))
        grapheme_root = torch.tensor(grapheme_root).long()
        vowel_diacritic = torch.tensor(vowel_diacritic).long()
        consonant_diacritic = torch.tensor(consonant_diacritic).long() 
        
        output_dict  = {
            'grapheme_root' : grapheme_root, 
            'vowel_diacritic' : vowel_diacritic, 
            'consonant_diacritic' : consonant_diacritic, 
            'image' : image
                       }

        return output_dict

    def __len__(self):
        return len(self.df)

Make train and validation datasets

In [73]:
batch_size = 16
num_workers = 1

In [74]:
idx = 0
train_fold_idx = train_labels['test_fold'] != idx
val_fold_idx = train_labels['test_fold'] == idx

In [75]:
aug_val = A.Compose([
    A.Normalize(mean=(0.485), std=(0.229))
],
        p=1.0)  
train_dataset = ImageDataset(df = train_df.loc[train_fold_idx, :], 
                             labels = train_labels.loc[train_fold_idx, :], 
                             transforms = aug_val
                            )
val_dataset = ImageDataset(df = train_df.loc[val_fold_idx, :],
                           labels = train_labels.loc[train_fold_idx, :], 
                           transforms = aug_val
                          )
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=True,
    shuffle=True
)
val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    pin_memory=True,
    shuffle=False   
    )

Callbacks for catalyst

In [111]:
from sklearn.metrics import recall_score
from catalyst.dl import Callback, RunnerState, MetricCallback, CallbackOrder, CriterionCallback

class TaskMetricCallback(Callback):
    '''
    Proposed metrics:
    import numpy as np
    import sklearn.metrics

    scores = []
    for component in ['grapheme_root', 'consonant_diacritic', 'vowel_diacritic']:
        y_true_subset = solution[solution[component] == component]['target'].values
        y_pred_subset = submission[submission[component] == component]['target'].values
        scores.append(sklearn.metrics.recall_score(
            y_true_subset, y_pred_subset, average='macro'))
    final_score = np.average(scores, weights=[2,1,1])
    '''

    def __init__(
        self, 
        input_key: str = ['grapheme_root', 'consonant_diacritic', 'vowel_diacritic'], 
        output_key: str = ['grapheme_root', 'consonant_diacritic', 'vowel_diacritic'],
        class_names: str = ['grapheme_root', 'consonant_diacritic', 'vowel_diacritic'],
        prefix: str = "taskmetric", 
        ignore_index=None
    ):
        super().__init__(CallbackOrder.Metric)
        self.metric_fn = lambda outputs, targets: recall_score(targets, outputs, average="macro")
        self.prefix = prefix
        self.output_key = output_key
        self.input_key = input_key
        self.class_names = class_names
        self.outputs = [[] for i in range(3)]
        self.targets = [[] for i in range(3)]

    def on_batch_end(self, state: RunnerState):
        
        for i in range(3):
            outputs = state.output[self.output_key[i]].detach().cpu().numpy()
            targets = state.input[self.input_key[i]].detach().cpu().numpy()
            #num_classes = outputs.shape[1]
            outputs = np.argmax(outputs, axis=1)
            #outputs = [np.eye(num_classes)[y] for y in outputs]
            #targets = [np.eye(num_classes)[y] for y in targets]
            self.outputs[i].extend(outputs)
            self.targets[i].extend(targets)

    def on_loader_start(self, state):
        self.outputs = [[] for i in range(3)]
        self.targets = [[] for i in range(3)]

    def on_loader_end(self, state):
        metric_name = self.prefix
        score_vec = []
        for i in range(3):
            targets = np.array(self.targets[i])
            outputs = np.array(self.outputs[i])
            metric = self.metric_fn(outputs, targets)
            score_vec.append(metric)
            state.metrics.epoch_values[state.loader_name][self.class_names[i]] = float(metric)
            
            
        state.metrics.epoch_values[state.loader_name][metric_name] = np.average(score_vec, weights=[2,1,1])

In [112]:
import torch.nn as nn

class ResidualBlock(nn.Module):
    def __init__(self,in_channels,out_channels,stride=1,kernel_size=3,padding=1,bias=False):
        super(ResidualBlock,self).__init__()
        self.cnn1 =nn.Sequential(
            nn.Conv2d(in_channels,out_channels,kernel_size,stride,padding,bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(True)
        )
        self.cnn2 = nn.Sequential(
            nn.Conv2d(out_channels,out_channels,kernel_size,1,padding,bias=False),
            nn.BatchNorm2d(out_channels)
        )
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,bias=False),
                nn.BatchNorm2d(out_channels)
            )
        else:
            self.shortcut = nn.Sequential()
    def forward(self,x):
        residual = x
        x = self.cnn1(x)
        x = self.cnn2(x)
        x += self.shortcut(residual)
        x = nn.ReLU(True)(x)
        return x
class ResNet18(nn.Module):    
    def __init__(self):
        super(ResNet18,self).__init__()
        
        self.block1 = nn.Sequential(
            nn.Conv2d(1,64,kernel_size=2,stride=2,padding=3,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )
        
        self.block2 = nn.Sequential(
            nn.MaxPool2d(1,1),
            ResidualBlock(64,64),
            ResidualBlock(64,64,2)
        )
        
        self.block3 = nn.Sequential(
            ResidualBlock(64,128),
            ResidualBlock(128,128,2)
        )
        self.block4 = nn.Sequential(
            ResidualBlock(128,256),
            ResidualBlock(256,256,2)
        )
        self.block5 = nn.Sequential(
            ResidualBlock(256,512),
            ResidualBlock(512,512,2)
        )
        
        self.avgpool = nn.AvgPool2d(2)
        # vowel_diacritic
        self.fc1 = nn.Linear(512,11)
        # grapheme_root
        self.fc2 = nn.Linear(512,168)
        # consonant_diacritic
        self.fc3 = nn.Linear(512,7)
        
    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        x1 = self.fc1(x)
        x2 = self.fc2(x)
        x3 = self.fc3(x)
        return({'vowel_diacritic':x1,
                'grapheme_root':x2,
                'consonant_diacritic':x3})
class ResNet34(nn.Module):    
    def __init__(self):
        super(ResNet34,self).__init__()
        
        self.block1 = nn.Sequential(
            nn.Conv2d(1,64,kernel_size=2,stride=2,padding=3,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True)
        )
        
        self.block2 = nn.Sequential(
            nn.MaxPool2d(1,1),
            ResidualBlock(64,64),
            ResidualBlock(64,64,2)
        )
        
        self.block3 = nn.Sequential(
            ResidualBlock(64,128),
            ResidualBlock(128,128,2)
        )
        
        self.block4 = nn.Sequential(
            ResidualBlock(128,256),
            ResidualBlock(256,256,2)
        )
        self.block5 = nn.Sequential(
            ResidualBlock(256,512),
            ResidualBlock(512,512,2)
        )
        
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        # vowel_diacritic
        self.fc1 = nn.Linear(512,11)
        # grapheme_root
        self.fc2 = nn.Linear(512,168)
        # consonant_diacritic
        self.fc3 = nn.Linear(512,7)
    def forward(self,x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = self.block5(x)
        x = self.avgpool(x)
        x = x.view(x.size(0),-1)
        x1 = self.fc1(x)
        x2 = self.fc2(x)
        x3 = self.fc3(x)
        return({'vowel_diacritic':x1,
                'grapheme_root':x2,
                'consonant_diacritic':x3})

In [113]:
model = ResNet34().cuda()

In [114]:
import collections
from catalyst.utils import set_global_seed
from catalyst.dl.runner import SupervisedRunner
from catalyst.dl.callbacks import CriterionCallback, CriterionAggregatorCallback

In [115]:
set_global_seed(42)

In [116]:
loaders = collections.OrderedDict()
loaders["train"] = train_loader
loaders["valid"] = val_loader
runner = SupervisedRunner(input_key='image',
                          input_target_key=None, 
                          output_key=None)

In [117]:
optimizer = torch.optim.AdamW(
    model.parameters(), 
    lr=3e-4, 
    weight_decay=0.001)  
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    factor=0.1, 
    patience=10) 

In [118]:
criterions_dict = {'vowel_diacritic_loss':torch.nn.CrossEntropyLoss(), 
                   'grapheme_root_loss':torch.nn.CrossEntropyLoss(),
                   'consonant_diacritic_loss':torch.nn.CrossEntropyLoss(),}

In [119]:
callbacks=[
    CriterionCallback(input_key='grapheme_root',
                      output_key='grapheme_root',
                      prefix='grapheme_root_loss',
                      criterion_key='grapheme_root_loss', multiplier=2.0),
    CriterionCallback(input_key='vowel_diacritic',
                      output_key='vowel_diacritic',
                      prefix='vowel_diacritic_loss',
                      criterion_key='vowel_diacritic_loss', 
                      multiplier=1.0),
    CriterionCallback(input_key='consonant_diacritic',
                      output_key='consonant_diacritic',
                      prefix='consonant_diacritic_loss',
                      criterion_key='consonant_diacritic_loss', 
                      multiplier=1.0),
    CriterionAggregatorCallback(prefix='loss',
                                loss_keys=['grapheme_root_loss',
                                           'vowel_diacritic_loss',
                                           'consonant_diacritic_loss']),
    TaskMetricCallback()]

In [ ]:
runner.train(
    model=model,
    main_metric='loss',
    minimize_metric=True,
    criterion=criterions_dict,
    optimizer=optimizer,
    callbacks=callbacks,
    loaders=loaders,
    logdir='./test_1',
    scheduler=scheduler,
    num_epochs=20,
    verbose=True)  

<IPython.core.display.Javascript object>



1/20 * Epoch (train):   0% 0/2823 [00:00<?, ?it/s]

1/20 * Epoch (train):   0% 0/2823 [00:00<?, ?it/s, consonant_diacritic_loss=2.046, grapheme_root_loss=10.372, loss=14.876, vowel_diacritic_loss=2.457]

1/20 * Epoch (train):   0% 1/2823 [00:00<12:41,  3.71it/s, consonant_diacritic_loss=2.046, grapheme_root_loss=10.372, loss=14.876, vowel_diacritic_loss=2.457]

1/20 * Epoch (train):   0% 1/2823 [00:00<12:41,  3.71it/s, consonant_diacritic_loss=2.012, grapheme_root_loss=10.243, loss=14.660, vowel_diacritic_loss=2.405]

1/20 * Epoch (train):   0% 2/2823 [00:00<10:18,  4.56it/s, consonant_diacritic_loss=2.012, grapheme_root_loss=10.243, loss=14.660, vowel_diacritic_loss=2.405]

1/20 * Epoch (train):   0% 2/2823 [00:00<10:18,  4.56it/s, consonant_diacritic_loss=1.735, grapheme_root_loss=10.522, loss=14.566, vowel_diacritic_loss=2.309]

1/20 * Epoch (train):   0% 3/2823 [00:00<08:52,  5.30it/s, consonant_diacritic_loss=1.735, grapheme_root_loss=10.522, loss=14.566, vowel_diacritic_loss=2.

1/20 * Epoch (train):   1% 25/2823 [00:03<05:34,  8.36it/s, consonant_diacritic_loss=1.555, grapheme_root_loss=9.533, loss=13.081, vowel_diacritic_loss=1.992]

1/20 * Epoch (train):   1% 25/2823 [00:03<05:34,  8.36it/s, consonant_diacritic_loss=1.227, grapheme_root_loss=9.797, loss=13.061, vowel_diacritic_loss=2.037]

1/20 * Epoch (train):   1% 26/2823 [00:03<05:21,  8.70it/s, consonant_diacritic_loss=1.227, grapheme_root_loss=9.797, loss=13.061, vowel_diacritic_loss=2.037]

1/20 * Epoch (train):   1% 26/2823 [00:03<05:21,  8.70it/s, consonant_diacritic_loss=1.011, grapheme_root_loss=10.266, loss=13.372, vowel_diacritic_loss=2.096]

1/20 * Epoch (train):   1% 27/2823 [00:03<05:26,  8.56it/s, consonant_diacritic_loss=1.011, grapheme_root_loss=10.266, loss=13.372, vowel_diacritic_loss=2.096]

1/20 * Epoch (train):   1% 27/2823 [00:03<05:26,  8.56it/s, consonant_diacritic_loss=0.836, grapheme_root_loss=9.838, loss=12.672, vowel_diacritic_loss=1.998] 

1/20 * Epoch (train):   1% 28/2823 [0

1/20 * Epoch (train):   2% 49/2823 [00:06<05:33,  8.32it/s, consonant_diacritic_loss=0.971, grapheme_root_loss=10.445, loss=13.581, vowel_diacritic_loss=2.164]

1/20 * Epoch (train):   2% 50/2823 [00:06<05:19,  8.67it/s, consonant_diacritic_loss=0.971, grapheme_root_loss=10.445, loss=13.581, vowel_diacritic_loss=2.164]

1/20 * Epoch (train):   2% 50/2823 [00:06<05:19,  8.67it/s, consonant_diacritic_loss=1.177, grapheme_root_loss=9.791, loss=12.672, vowel_diacritic_loss=1.705] 

1/20 * Epoch (train):   2% 51/2823 [00:06<05:24,  8.54it/s, consonant_diacritic_loss=1.177, grapheme_root_loss=9.791, loss=12.672, vowel_diacritic_loss=1.705]

1/20 * Epoch (train):   2% 51/2823 [00:06<05:24,  8.54it/s, consonant_diacritic_loss=1.079, grapheme_root_loss=9.642, loss=12.619, vowel_diacritic_loss=1.897]

1/20 * Epoch (train):   2% 52/2823 [00:06<05:29,  8.41it/s, consonant_diacritic_loss=1.079, grapheme_root_loss=9.642, loss=12.619, vowel_diacritic_loss=1.897]

1/20 * Epoch (train):   2% 52/2823 [0

1/20 * Epoch (train):   3% 74/2823 [00:08<05:21,  8.56it/s, consonant_diacritic_loss=1.356, grapheme_root_loss=9.585, loss=12.308, vowel_diacritic_loss=1.367]

1/20 * Epoch (train):   3% 75/2823 [00:08<05:23,  8.49it/s, consonant_diacritic_loss=1.356, grapheme_root_loss=9.585, loss=12.308, vowel_diacritic_loss=1.367]

1/20 * Epoch (train):   3% 75/2823 [00:09<05:23,  8.49it/s, consonant_diacritic_loss=1.484, grapheme_root_loss=8.910, loss=11.559, vowel_diacritic_loss=1.165]

1/20 * Epoch (train):   3% 76/2823 [00:09<05:27,  8.40it/s, consonant_diacritic_loss=1.484, grapheme_root_loss=8.910, loss=11.559, vowel_diacritic_loss=1.165]

1/20 * Epoch (train):   3% 76/2823 [00:09<05:27,  8.40it/s, consonant_diacritic_loss=1.389, grapheme_root_loss=8.669, loss=12.461, vowel_diacritic_loss=2.402]

1/20 * Epoch (train):   3% 77/2823 [00:09<05:30,  8.31it/s, consonant_diacritic_loss=1.389, grapheme_root_loss=8.669, loss=12.461, vowel_diacritic_loss=2.402]

1/20 * Epoch (train):   3% 77/2823 [00:0

...

1/20 * Epoch (train):  15% 416/2823 [00:43<04:30,  8.89it/s, consonant_diacritic_loss=0.353, grapheme_root_loss=9.464, loss=10.739, vowel_diacritic_loss=0.921]

1/20 * Epoch (train):  15% 417/2823 [00:43<04:38,  8.63it/s, consonant_diacritic_loss=0.353, grapheme_root_loss=9.464, loss=10.739, vowel_diacritic_loss=0.921]

1/20 * Epoch (train):  15% 417/2823 [00:43<04:38,  8.63it/s, consonant_diacritic_loss=0.756, grapheme_root_loss=8.324, loss=10.264, vowel_diacritic_loss=1.183]

1/20 * Epoch (train):  15% 418/2823 [00:43<04:45,  8.44it/s, consonant_diacritic_loss=0.756, grapheme_root_loss=8.324, loss=10.264, vowel_diacritic_loss=1.183]

1/20 * Epoch (train):  15% 418/2823 [00:43<04:45,  8.44it/s, consonant_diacritic_loss=0.675, grapheme_root_loss=8.837, loss=9.985, vowel_diacritic_loss=0.474] 

1/20 * Epoch (train):  15% 419/2823 [00:43<04:44,  8.44it/s, consonant_diacritic_loss=0.613, grapheme_root_loss=8.704, loss=10.060, vowel_diacritic_loss=0.743]

1/20 * Epoch (train):  15% 420/282

1/20 * Epoch (train):  16% 444/2823 [00:46<04:10,  9.50it/s, consonant_diacritic_loss=0.822, grapheme_root_loss=8.380, loss=10.005, vowel_diacritic_loss=0.803]

1/20 * Epoch (train):  16% 445/2823 [00:46<04:10,  9.50it/s, consonant_diacritic_loss=0.648, grapheme_root_loss=8.830, loss=10.047, vowel_diacritic_loss=0.569]

1/20 * Epoch (train):  16% 446/2823 [00:46<04:24,  9.00it/s, consonant_diacritic_loss=0.648, grapheme_root_loss=8.830, loss=10.047, vowel_diacritic_loss=0.569]

1/20 * Epoch (train):  16% 446/2823 [00:46<04:24,  9.00it/s, consonant_diacritic_loss=0.525, grapheme_root_loss=8.953, loss=10.063, vowel_diacritic_loss=0.585]

1/20 * Epoch (train):  16% 447/2823 [00:46<04:31,  8.75it/s, consonant_diacritic_loss=0.525, grapheme_root_loss=8.953, loss=10.063, vowel_diacritic_loss=0.585]

1/20 * Epoch (train):  16% 447/2823 [00:46<04:31,  8.75it/s, consonant_diacritic_loss=0.694, grapheme_root_loss=8.140, loss=9.695, vowel_diacritic_loss=0.862] 

1/20 * Epoch (train):  16% 448/282

1/20 * Epoch (train):  17% 474/2823 [00:49<04:04,  9.62it/s, consonant_diacritic_loss=0.529, grapheme_root_loss=8.333, loss=9.333, vowel_diacritic_loss=0.471]

1/20 * Epoch (train):  17% 474/2823 [00:49<04:04,  9.62it/s, consonant_diacritic_loss=0.710, grapheme_root_loss=7.746, loss=9.185, vowel_diacritic_loss=0.728]

1/20 * Epoch (train):  17% 475/2823 [00:49<04:03,  9.62it/s, consonant_diacritic_loss=1.378, grapheme_root_loss=8.065, loss=10.166, vowel_diacritic_loss=0.723]

1/20 * Epoch (train):  17% 476/2823 [00:49<04:17,  9.10it/s, consonant_diacritic_loss=1.378, grapheme_root_loss=8.065, loss=10.166, vowel_diacritic_loss=0.723]

1/20 * Epoch (train):  17% 476/2823 [00:49<04:17,  9.10it/s, consonant_diacritic_loss=0.845, grapheme_root_loss=8.459, loss=10.325, vowel_diacritic_loss=1.021]

1/20 * Epoch (train):  17% 477/2823 [00:49<04:26,  8.79it/s, consonant_diacritic_loss=0.845, grapheme_root_loss=8.459, loss=10.325, vowel_diacritic_loss=1.021]

1/20 * Epoch (train):  17% 477/2823 

1/20 * Epoch (train):  18% 503/2823 [00:52<04:29,  8.61it/s, consonant_diacritic_loss=1.315, grapheme_root_loss=9.057, loss=10.992, vowel_diacritic_loss=0.620]

1/20 * Epoch (train):  18% 504/2823 [00:52<04:02,  9.55it/s, consonant_diacritic_loss=1.315, grapheme_root_loss=9.057, loss=10.992, vowel_diacritic_loss=0.620]

1/20 * Epoch (train):  18% 504/2823 [00:52<04:02,  9.55it/s, consonant_diacritic_loss=1.099, grapheme_root_loss=8.155, loss=9.732, vowel_diacritic_loss=0.478] 

1/20 * Epoch (train):  18% 505/2823 [00:52<04:02,  9.55it/s, consonant_diacritic_loss=0.687, grapheme_root_loss=7.700, loss=9.553, vowel_diacritic_loss=1.166]

1/20 * Epoch (train):  18% 506/2823 [00:52<04:14,  9.09it/s, consonant_diacritic_loss=0.687, grapheme_root_loss=7.700, loss=9.553, vowel_diacritic_loss=1.166]

1/20 * Epoch (train):  18% 506/2823 [00:52<04:14,  9.09it/s, consonant_diacritic_loss=1.020, grapheme_root_loss=8.171, loss=10.273, vowel_diacritic_loss=1.083]

1/20 * Epoch (train):  18% 507/2823 

1/20 * Epoch (train):  19% 532/2823 [00:55<04:28,  8.53it/s, consonant_diacritic_loss=0.850, grapheme_root_loss=8.760, loss=10.565, vowel_diacritic_loss=0.954]

1/20 * Epoch (train):  19% 533/2823 [00:55<04:18,  8.87it/s, consonant_diacritic_loss=0.850, grapheme_root_loss=8.760, loss=10.565, vowel_diacritic_loss=0.954]

1/20 * Epoch (train):  19% 533/2823 [00:55<04:18,  8.87it/s, consonant_diacritic_loss=0.674, grapheme_root_loss=8.859, loss=10.824, vowel_diacritic_loss=1.291]

1/20 * Epoch (train):  19% 534/2823 [00:55<04:18,  8.87it/s, consonant_diacritic_loss=0.287, grapheme_root_loss=8.988, loss=10.281, vowel_diacritic_loss=1.007]

1/20 * Epoch (train):  19% 535/2823 [00:55<04:01,  9.48it/s, consonant_diacritic_loss=0.287, grapheme_root_loss=8.988, loss=10.281, vowel_diacritic_loss=1.007]

1/20 * Epoch (train):  19% 535/2823 [00:55<04:01,  9.48it/s, consonant_diacritic_loss=0.803, grapheme_root_loss=7.812, loss=9.515, vowel_diacritic_loss=0.900] 

1/20 * Epoch (train):  19% 536/282